In [151]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

In [11]:
analysis_date = dt.date(2021, 11, 30)

In [10]:
dt.datetime.strptime('11-30-2021', '%m-%d-%Y').date()

datetime.date(2021, 11, 30)

In [ ]:
## 

In [13]:
analysis_date.strftime('%Y-%m-%d')

'2021-11-30'

In [16]:
date_str = analysis_date.strftime('%Y-%m-%d')

In [18]:
next_str = (analysis_date + dt.timedelta(days = 1)).strftime('%Y-%m-%d')

In [17]:
samtrans_itp_id = 290

In [ ]:
## todo check for multiple url numbers...

In [20]:
f"gs://gtfs-data/rt-processed/vehicle_positions/vp_{date_str}_{samtrans_itp_id}_0.parquet"

'gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-11-30_290_0.parquet'

In [21]:
f"gs://gtfs-data/rt-processed/vehicle_positions/vp_{next_str}_{samtrans_itp_id}_0.parquet"

'gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_290_0.parquet'

In [40]:
%%sql -o samtrans_positions

# get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
SELECT *
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-11-30_290_0.parquet' ##utc, must bracket 2 days for 1 day pacific
    OR _FILE_NAME='gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_290_0.parquet'
ORDER BY header_timestamp

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,290,0,1,2912,11344573-136-Blocks-Weekday-54-SDon-1111000,1638230373,37.470440,-122.433710,1638230410
1,290,0,10,630,11344643-136-Blocks-Weekday-54-SDon-1100100,1638230373,37.461860,-122.199295,1638230410
2,290,0,100,818,11031714-136-Blocks-Weekday-54,1638230373,37.617565,-122.408356,1638230410
3,290,0,101,609,11037713-136-Blocks-Weekday-54,1638230373,37.669320,-122.471020,1638230410
4,290,0,102,409,11031571-136-Blocks-Weekday-54,1638230373,37.707930,-122.454830,1638230410
...,...,...,...,...,...,...,...,...,...
474976,290,0,83,819,11031584-136-Blocks-Weekday-54,1638386133,37.664288,-122.448570,1638386151
474977,290,0,84,901,11031483-136-Blocks-Weekday-54,1638386133,37.685078,-122.466340,1638386151
474978,290,0,85,813,11031588-136-Blocks-Weekday-54,1638386133,37.654964,-122.433815,1638386151
474979,290,0,86,617,11037708-136-Blocks-Weekday-54,1638386133,37.666320,-122.451645,1638386151


In [41]:
## set system time
os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [42]:
samtrans_positions.vehicle_timestamp = samtrans_positions.vehicle_timestamp.apply(convert_ts)
samtrans_positions.header_timestamp = samtrans_positions.header_timestamp.apply(convert_ts)

In [43]:
samtrans_positions.vehicle_timestamp.max() ## TODO assert covers desired day

Timestamp('2021-12-01 11:15:33')

In [44]:
samtrans_positions.vehicle_timestamp.min()

Timestamp('2021-11-29 15:59:33')

In [45]:
date_str

'2021-11-30'

In [46]:
next_str

'2021-12-01'

In [47]:
min_date, max_date = (date_str, next_str)

samtrans_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
    >> filter(_.calitp_itp_id == samtrans_itp_id)
    >> filter(_.service_date == date_str)
    >> filter(_.calitp_extracted_at == _.calitp_extracted_at.max())
    >> filter(_.is_in_service == True)
    >> select(_.trip_key, _.service_date)
    >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
    >> select(_.calitp_itp_id, _.calitp_url_number, _.service_date,
              _.trip_key, _.trip_id, _.route_id, _.direction_id,
              _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
    >> collect()
    )

/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self.connectable.execute(*args, **kwargs)


In [53]:
samtrans_trips.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at
0,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21
1,290,1,2021-11-30,-5135734635980113023,11038139-136-Blocks-Weekday-54,256,0,2560186,2021-10-12,2022-01-05
2,290,1,2021-11-30,-5532324825960195079,11032605-136-Blocks-Weekday-54,122,0,1220566,2021-10-24,2021-12-21


In [54]:
samtrans_trips >> count(_.trip_id) >> mutate(count_n = _.n.sum()) ## assert something like this (encouraging)

,trip_id,n,count_n
0,11030731-136-Blocks-Weekday-54,1,1451
1,11030732-136-Blocks-Weekday-54,1,1451
2,11030733-136-Blocks-Weekday-54,1,1451
3,11030734-136-Blocks-Weekday-54,1,1451
4,11030735-136-Blocks-Weekday-54,1,1451
...,...,...,...
1446,11344692-136-Blocks-Weekday-54-SDon,1,1451
1447,11344693-136-Blocks-Weekday-54-SDon-1101100,1,1451
1448,11344695-136-Blocks-Weekday-54-SDon-1100100,1,1451
1449,11344697-136-Blocks-Weekday-54-SDon-1100100,1,1451


In [ ]:
stop_times = (tbl.gtfs_schedule.stop_times()
     >> filter(_.calitp_itp_id == 170)
     # >> filter(_.trip_id == '9288623')
     >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
     >> arrange(_.stop_sequence)
     >> collect()
    )

In [107]:
min_date, max_date = (date_str, next_str)

sam_trips_query = (tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
    >> filter(_.calitp_itp_id == samtrans_itp_id)
    >> filter(_.service_date == date_str)
    >> filter(_.calitp_extracted_at == _.calitp_extracted_at.max())
    >> filter(_.is_in_service == True)
    >> select(_.trip_key, _.service_date)
    )

In [108]:
sam_trips_ix_query = (sam_trips_query
    >> inner_join(_, tbl.views.gtfs_schedule_index_feed_trip_stops(), on = 'trip_key')
    )

In [109]:
sam_st_query = (tbl.views.gtfs_schedule_dim_stop_times()
    >> filter(_.calitp_itp_id == samtrans_itp_id)
    )

In [110]:
sam_st = (sam_trips_ix_query
    >> inner_join(_, sam_st_query, on = 'stop_time_key')
    >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
    >> arrange(_.stop_sequence)
    >> collect()
    )

* it kind of seems that once you've got a key, filtering extracted/deleted is bad news...

In [114]:
stops = (tbl.views.gtfs_schedule_dim_stops()
     >> filter(_.calitp_itp_id == samtrans_itp_id)
     >> distinct(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
              _.stop_lat, _.stop_lon, _.stop_name, _.stop_key)
     # >> collect()
     # >> distinct(_.stop_key, _keep_all=True)
     >> inner_join(_, sam_trips_ix_query >> distinct(_.stop_key), on = 'stop_key')
     >> collect()
    )

stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                        crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [124]:
stops.head(3)

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,stop_key,geometry
0,290,1,332218,37.706419,-122.459780,Mission St & Flournoy St,8328312696755447011,POINT (-216540.225 -31646.559)
1,290,1,341153,37.536236,-122.297005,El Camino Real & Hillsdale Blvd,-5105536355019630068,POINT (-202671.972 -50919.009)
2,290,1,341236,37.553320,-122.290693,S Norfolk St & Fashion Island Blvd,3584246796067643826,POINT (-202069.271 -49033.894)


In [123]:
stops >> filter(_.stop_name.str.contains('Ruiss'))

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,stop_key,geometry
420,290,1,315017,37.483802,-122.446712,Hwy 1 & Ruisseau Francais Ave,-8064029110528363578,POINT (-216028.655 -56415.420)
723,290,1,315018,37.483849,-122.446537,Hwy 1 & Ruisseau Francais Ave,-6888104359049458766,POINT (-216013.072 -56410.596)
1956,290,1,315018,37.483849,-122.446537,Hwy 1 & Ruisseau Francais Ave,5626421419577972445,POINT (-216013.072 -56410.596)
3187,290,1,315017,37.483802,-122.446712,Hwy 1 & Ruisseau Francais Ave,7985544784696432888,POINT (-216028.655 -56415.420)
3520,290,1,315017,37.483802,-122.446712,Hwy 1 & Ruisseau Francais Ave,-1355362411902854211,POINT (-216028.655 -56415.420)
4211,290,1,315018,37.483849,-122.446537,Hwy 1 & Ruisseau Francais Ave,-4832955083364989897,POINT (-216013.072 -56410.596)


* we seem to have 3 rows per stop with different keys... (can't drop if we need keys to match with trips...)

In [131]:
trips_positions_joined = (samtrans_trips
                        >> inner_join(_, (samtrans_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id'])
                       )

In [133]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [134]:
trips_positions_joined.head(3)

,calitp_itp_id,calitp_url_number_x,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,calitp_url_number_y,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
0,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21,0,70,509,2021-11-29 15:59:33,37.534653,-122.300766,2021-11-29 16:00:10,POINT (-203008.021 -51086.885)
1,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21,0,68,509,2021-11-29 16:00:14,37.537376,-122.311630,2021-11-29 16:00:30,POINT (-203959.037 -50760.956)
2,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21,0,68,509,2021-11-29 16:00:14,37.537376,-122.311630,2021-11-29 16:00:50,POINT (-203959.037 -50760.956)


In [137]:
st_routelines = shared_utils.geography_utils.make_routes_shapefile([samtrans_itp_id], ## samtrans
                                shared_utils.geography_utils.CA_NAD83Albers)

In [143]:
# shared_utils.utils.geoparquet_gcs_export(st_routelines, GCS_FILE_PATH, 'sam_routelines')

In [141]:
sam_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}sam_routelines.parquet')

In [142]:
sam_routelines

,calitp_itp_id,calitp_url_number,shape_id,geometry
0,290,0,1100224,"LINESTRING (-217339.785 -31678.594, -217339.78..."
1,290,1,1100236,"LINESTRING (-217339.785 -31678.594, -217339.78..."
2,290,0,1100237,"LINESTRING (-220663.991 -43864.472, -220663.99..."
3,290,1,1100256,"LINESTRING (-218302.184 -44113.771, -218302.18..."
4,290,1,1120134,"LINESTRING (-217098.080 -34074.660, -217098.08..."
...,...,...,...,...
438,290,1,SamTrans79:ECR1064,"LINESTRING (-216685.505 -31709.510, -216679.32..."
439,290,1,SamTrans79:ECR1065,"LINESTRING (-216685.505 -31709.510, -216679.32..."
440,290,1,SamTrans79:ECR1072,"LINESTRING (-191349.232 -61446.984, -191332.05..."
441,290,1,SamTrans79:FLXP0056,"LINESTRING (-220663.991 -43864.472, -220670.53..."


In [167]:
# trips_positions_joined >> filter(_.route_id == 'ECR')

In [170]:
example_trip = (trips_positions_joined
 >> filter(_.trip_key == 9182511101216666422)
)

In [191]:
importlib.reload(rt)

<module 'rt_analysis2' from '/home/jovyan/data-analyses/rt_delay/rt_analysis2.py'>

In [192]:
# example_trip[['calitp_url_number_x', 'calitp_url_number_y']] ## RT and static numbers, these need not match?

In [193]:
## gums up for 294...

In [194]:
example_vp_trip = rt.VehiclePositionsInterpolator(example_trip, sam_routelines)

sc_called
(422, 10)
check location data for trip 9182511101216666422
sc_called
(111, 16)


In [198]:
# example_vp_trip.detailed_speed_map()